In [1]:
!pip install fastkml
!pip install fastkml lxml pandas openpyxl utm
!pip install pyproj

from fastkml import kml
from google.colab import files
import zipfile, os, glob
import xml.etree.ElementTree as ET
import utm
from lxml import etree
import pandas as pd
import shutil
import pyproj
import re
from pyproj import CRS, Transformer



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.8 MB/s eta 0:00:00


Erro: [Errno 2] No such file or directory: 'CTO-NTL-02059.kmz'


In [ ]:
def descompactar_kmz(kmz_path):
    caminho_extraido = 'temp_kmz'
    if os.path.exists(caminho_extraido):
        shutil.rmtree(caminho_extraido)
    os.makedirs(caminho_extraido)

    with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
        zip_ref.extractall(caminho_extraido)

    kml_files = [f for f in os.listdir(caminho_extraido) if f.endswith('.kml')]
    if len(kml_files) == 1:
        return os.path.join(caminho_extraido, kml_files[0])
    else:
        raise ValueError("Nenhum ou múltiplos arquivos KML encontrados.")

# Função para extrair os placemarks
def extrair_placemarks_de_kml(caminho_kml):
    placemarks = []
    with open(caminho_kml, 'rb') as file:
        conteudo = file.read()

    root = etree.fromstring(conteudo)
    for elem in root.iter():
        if 'Placemark' in elem.tag:
            placemarks.append(elem)
    return placemarks

# Função para processar a descrição e extrair os campos desejados (alterar de acordo com a descrição do placemark)
def processar_descricao(texto):
    campos = {
        "Código do poste (S/N)": "",
        "Altura/esforço": "",
        "Média tensão (SIM/NÃO)": "",
        "Baixa tensão (SIM/NÃO)": "",
        "Ilum. Pública (SIM/NÃO)": "",
        "Transformador (SIM/NÃO)": "",
        "Chave fusível (SIM/NÃO)": "",
        "Operadoras": "",
        "Metragem do ponto anterior até aqui": ""
    }

    # Inicializar campos derivados
    altura = ""
    esforco = ""
    geometria = ""

    if texto:
        linhas = texto.strip().split('\n')

        # Procurar Geometria do poste por palavras-chave no texto
        for linha in linhas:
            if any(g in linha.upper() for g in ["DTT", "DT", "CC"]):
                if "DTT" in linha.upper():
                    geometria = "DTT"
                elif "DT" in linha.upper():
                    geometria = "DT"
                elif "CC" in linha.upper():
                    geometria = "CC"

        # Demais campos
        for linha in linhas:
            for chave in campos.keys():
                if chave.lower() in linha.lower():
                    partes = linha.split(':', 1)
                    if len(partes) == 2:
                        valor = partes[1].strip().strip('"')
                        campos[chave] = valor
                        # Separar altura e esforço
                        if chave == "Altura/esforço" and '/' in valor:
                            partes_alt = valor.split('/')
                            if len(partes_alt) == 2:
                                altura = partes_alt[0].strip()
                                esforco = partes_alt[1].strip()

    return {
        "Geometria do poste": geometria,
        "Código do poste (S/N)": campos["Código do poste (S/N)"],
        "Altura/esforço": campos["Altura/esforço"],
        "Altura": altura,
        "Esforço": esforco,
        "Média tensão (SIM/NÃO)": campos["Média tensão (SIM/NÃO)"],
        "Baixa tensão (SIM/NÃO)": campos["Baixa tensão (SIM/NÃO)"],
        "Ilum. Pública (SIM/NÃO)": campos["Ilum. Pública (SIM/NÃO)"],
        "Transformador (SIM/NÃO)": campos["Transformador (SIM/NÃO)"],
        "Chave fusível (SIM/NÃO)": campos["Chave fusível (SIM/NÃO)"],
        "Operadoras": campos["Operadoras"],
        "Metragem do ponto anterior até aqui": campos["Metragem do ponto anterior até aqui"]
    }

# Função principal
def processar_kmz(kmz_path, arquivo_excel_saida):
    try:
        caminho_kml = descompactar_kmz(kmz_path)
        print(f"Arquivo KML extraído de {kmz_path}: {caminho_kml}")

        placemarks = extrair_placemarks_de_kml(caminho_kml)
        print(f"{len(placemarks)} placemarks encontrados.")

        dados = []
        for placemark in placemarks:
            nome = placemark.find(".//{http://www.opengis.net/kml/2.2}name")
            descricao = placemark.find(".//{http://www.opengis.net/kml/2.2}description")
            coordenadas = placemark.find(".//{http://www.opengis.net/kml/2.2}coordinates")

            if nome is not None and coordenadas is not None:
                partes = coordenadas.text.strip().split(',')
                if len(partes) >= 2:
                    lon = float(partes[0])
                    lat = float(partes[1])

                    lon_decimal = float(f"{lon:.6f}")
                    lat_decimal = float(f"{lat:.6f}")

                    easting, northing, zone_number, zone_letter = utm.from_latlon(lat, lon)

                    campos_descritos = processar_descricao(descricao.text if descricao is not None else "")

                    dados.append({
                        "Nome": nome.text,
                        "Longitude (graus decimais)": lon_decimal,
                        "Latitude (graus decimais)": lat_decimal,
                        "UTM Leste (m)": round(easting, 2),
                        "UTM Norte (m)": round(northing, 2),
                        "Zona UTM": f"{zone_number}{zone_letter}",
                        **campos_descritos
                    })

        if dados:
            df = pd.DataFrame(dados)
            df.to_excel(arquivo_excel_saida, index=False)
            print(f"Excel salvo como: {arquivo_excel_saida}")
        else:
            print("Nenhum dado encontrado nos placemarks.")

    except Exception as e:
        print(f"Erro: {e}")

# EXECUÇÃO (trocar o teste.kmz de acordo com o nome do seu arquivo .kmz )
processar_kmz('teste.kmz', 'teste_extraidos.xlsx')

Arquivo KML extraído de teste.kmz: temp_kmz/doc.kml
149 placemarks encontrados.
Excel salvo como: teste_extraidos.xlsx
